In [4]:
import json
import pandas as pd


def read_experiment_data(file_path):
    """读取实验数据文件"""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def extract_experiment_info(experiments):
    """提取实验信息"""
    results = []

    for exp in experiments:
        config = exp.get("config", {})
        result = exp.get("result", {})

        # 提取所需的参数
        edge_cloud_bandwidth = config.get("edge_cloud_bandwidth", 0)
        edge_end_bandwidth = config.get("edge_end_bandwidth", 0)
        eval_mode = result.get("eval_mode", "")
        target_forward_times = result.get("target_forward_times", 0)
        wall_time = result.get("wall_time", 0)
        communication_time = result.get("communication_time", 0)
        edge_cloud_data_bytes = result.get("edge_cloud_data_bytes", 0)
        throughput = result.get("throughput", 0)
        comm_energy = result.get("comm_energy", 0)

        results.append(
            {
                "Edge-Cloud 带宽 (Mbps)": edge_cloud_bandwidth,
                "Edge-End 带宽 (Mbps)": edge_end_bandwidth,
                "方法": eval_mode,
                "目标模型 Forward 次数": target_forward_times,
                "挂钟时间 (s)": round(wall_time, 2),
                "通信时间 (s)": round(communication_time, 2),
                "Edge-Cloud 数据传输量 (bytes)": edge_cloud_data_bytes,
                "Token 输出速率 (tokens/s)": round(throughput, 2),
                "通信能量": comm_energy,
            }
        )

    return results


def generate_markdown_table(data):
    """生成 markdown 表格"""
    df = pd.DataFrame(data)

    # 按方法分组，然后按Edge-Cloud带宽降序排序
    df_sorted = df.sort_values(
        ["方法", "Edge-Cloud 带宽 (Mbps)"], ascending=[True, False]
    )

    # 生成 markdown 表格
    markdown_table = df_sorted.to_markdown(index=False, tablefmt="github")
    return markdown_table


def generate_grouped_tables(data):
    """按方法分组生成多个表格"""
    df = pd.DataFrame(data)

    # 按方法分组
    grouped = df.groupby("方法")

    all_tables = []

    for method, group_df in grouped:
        # 按Edge-Cloud带宽降序排序
        group_df_sorted = group_df.sort_values(
            "Edge-Cloud 带宽 (Mbps)", ascending=False
        )

        # 生成表格
        table = group_df_sorted.to_markdown(index=False, tablefmt="github")

        all_tables.append(f"## {method}\n\n{table}\n")

    return "\n".join(all_tables)


def main():
    # 读取实验数据
    file_path = "experiment_summary_20250920_145859.json"
    experiments = read_experiment_data(file_path)

    # 提取实验信息
    experiment_data = extract_experiment_info(experiments)

    # 生成统一排序的表格
    print("# 实验结果对比表（按方法和带宽排序）")
    print()
    unified_table = generate_markdown_table(experiment_data)
    print(unified_table)
    print("\n" + "=" * 80 + "\n")

    # 生成按方法分组的表格
    print("# 实验结果对比表（按方法分组）")
    print()
    grouped_tables = generate_grouped_tables(experiment_data)
    print(grouped_tables)

    # 保存到文件
    with open("experiment_results_table.md", "w", encoding="utf-8") as f:
        f.write("# 实验结果对比表（按方法和带宽排序）\n\n")
        f.write(unified_table)
        f.write("\n\n" + "=" * 80 + "\n\n")
        f.write("# 实验结果对比表（按方法分组）\n\n")
        f.write(grouped_tables)

    print("\n表格已保存到 experiment_results_table.md 文件中")


if __name__ == "__main__":
    main()

# 实验结果对比表（按方法和带宽排序）

|   Edge-Cloud 带宽 (Mbps) |   Edge-End 带宽 (Mbps) | 方法                 |   目标模型 Forward 次数 |   挂钟时间 (s) |   通信时间 (s) |   Edge-Cloud 数据传输量 (bytes) |   Token 输出速率 (tokens/s) |   通信能量 |
|--------------------------|------------------------|----------------------|-------------------------|----------------|----------------|---------------------------------|-----------------------------|------------|
|                    34.6  |                  563   | dist_spec            |                    5327 |         639.63 |           8.87 |                        38381416 |                       32.02 |          0 |
|                    25    |                  350   | dist_spec            |                    5327 |         637.92 |          12.28 |                        38381416 |                       32.1  |          0 |
|                    15    |                  200   | dist_spec            |                    5327 |         652.72 |          20.47 |                    